In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
df_minjust = pd.read_csv('data/Minjust2018.csv', 
                 header=None, 
                 usecols=[4, 27, 28, 33])
df_minjust.fillna('', inplace=True)
df_minjust.rename(columns={4: 'company_name', 
                           27: 'director', 
                           28: 'activity', 
                           33: 'founders'},
                  inplace = True)

In [ ]:
df_minjust

In [ ]:
df_decl = pd.read_csv('data/declaration.csv', 
                      header=None, 
                      usecols=[1, 2, 3, 4, 5])
df_decl.fillna('', inplace=True)
df_decl.rename(columns={1: 'surname', 
                        2: 'name', 
                        3: 'patronymic', 
                        4: 'workplace',
                        5: 'position'},
                  inplace = True)

In [ ]:
df_decl

In [ ]:
columns = ['company_name', 'director', 'activity', 'founders', 'surname', 'name', 'patronymic', 'workplace', 'position']
result = pd.DataFrame(columns=columns)
result1 = pd.DataFrame(columns=columns)
result2 = pd.DataFrame(columns=columns)

In [ ]:
for index, row in df_decl.loc[: 10].iterrows():
    search = search_df(df_minjust, 'founders', child_regex(row['surname'], row['name'])).copy()
    for column in df_decl.columns:
        search[column] = row[column]
    result = result.append(search, ignore_index=True)
    

In [ ]:
result

In [ ]:
def search_df(df, column, pattern):
    return df.loc[df[column].str.contains(pattern)]
    
def child_regex(surname, name):
    pattern = re.compile(surname +
                         r'а?\s+\w+\s+' + # First name
                         remove_vowels(name) + # Patronymic
                         r'|' + 
                         r'\w+\s+' + # First name
                         remove_vowels(name) + r'\w+\s+' + # Patronymic
                         surname,
                         re.IGNORECASE)
    return pattern

def remove_vowels(name):
    while True:
        if name[-1] not in ['а', 'о', 'и', 'е', 'ё', 'э', 'ы', 'у', 'ю', 'я', 'й']:
            break
        else:
            name = name[: -1]
    return name

In [ ]:
for index, row in df_decl.loc[355: 425].iterrows():
    search = search_df(df_minjust, 'founders', child_regex(row['surname'], row['name'])).copy()
    for column in df_decl.columns:
        search[column] = row[column]
    result1 = result1.append(search, ignore_index=True)

In [ ]:
result1

In [ ]:
def brother_regex(surname, patronymic):
    pattern = re.compile(surname +
                         r'\s+\w+\s+' + # First name
                         patronymic + # Patronymic
                         r'|' + 
                         r'\w+\s+' +
                         patronymic + r'\s+' + # Patronymic
                         surname,
                         re.IGNORECASE)
    return pattern

def count_letters(words):
    return pd.Series([len(word) for word in words])

In [ ]:
for index, row in df_decl.loc[355: 425].iterrows():
    search = search_df(df_minjust, 'director', brother_regex(row['surname'], row['patronymic'])).copy()
    for column in df_decl.columns:
        search[column] = row[column]
    result2 = result2.append(search, ignore_index=True)

In [ ]:
result2

In [ ]:
result.to_pickle('result1.pkl')

In [ ]:
result.to_pickle('result2.pkl')